# 8Pay valuation model

### Current Token Setup
8pay collects a fee of 1 \% of all payments made through their system. Half of those fees will go to 8pay and half will be redistributed among all stakers of the 8pay token on e.g. a weekly basis. As such, the 8pay token currently functions as a security token and its value depends entirely on the future cashflow provided by fees via staking, since its additional function of working as a means of exchange is satisfied in a superior fashion by .

### Prerequisits
To calculate the value of the entire token supply, we have to figure out the transaction volume (number of transactions times average transaction value) which serves as the basis from which fees are taken, along with a discount rate. The discount rate can include a lot of other factors, such as real interest rate (i.e. nominal interest rates minus inflation rate) or risk - and perhaps it should even take the yield curve into consideration.

### Modelling
The implemented model sums the discounted cashflows over the given time horizon and assumes that after that time horizon has been reached, cashflows remaining constant at the amount of the time horizon crossing. Compounding is done on a "per period" basis, with periods being either yearly, quarterly, monthly, or weekly. Rates are adjusted to always yield the specified discount rate on an annual basis. To model the transaction count, I chose a parameterized S-shaped curve (mid, slope, limit) with initial exponential growth that eventually saturates.

### Applicability
The main issues with this model are of course the uncertainty involved in the entire venture itself, the projection of the number of 8pay transactions and their average value, along with the interest rates that come to bear.

### Next Steps
This model is mainly intended as a toy model aimed at illustrating the approach that ought to be taken to calculate the fair value of the 8pay token at any point in time, given known data. To get to a better evaluation for investors, one could use this model to generate different scenarios (e.g. small, medium, and large scale adoption), assign probabilities to these scenarios and then do an expected value calculation based on those numbers.

In [1]:
import scipy.integrate as integrate
import numpy
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

We use a logistic function for our S-shaped curve, though any curve shape will work and one should look at references classes of other projects to estimate what a reasonable adoption curve might look like.

In [2]:
logistic = lambda x, x0, k, L: L/(1+numpy.exp(-k*(x-x0)))

Integrating the given span of time to find e.g. transaction volume yields more robust results than just doing e.g. linear interpolation.

In [3]:
average = lambda func, interval: integrate.quad(func, interval[0], interval[1])[0]

Back of the envelope calculation on the maximum number of available token transactions on the Ethereum network given its current technological state and the assumption of about 100k gas for a single token transfer.

In [11]:
blockGasLimit = 8000000
gasPerTransaction = 100000
blocksPerMinute = 4
annualMaxTransactions = numpy.floor(blockGasLimit/gasPerTransaction)*blocksPerMinute*60*24*365
printmd(f'**{annualMaxTransactions/10**6}M**')

**168.192M**

Primary function to calculate, sum, and plot the net present value of all future cashflows.

Parameters:
* timeHorizon: number of years to be calculated - transaction volume past the time horizon is assumed to remain constant in perpetuity
* periodsPerYear: length of the staking interval. also determines the effective discount rate.
* discountRate: annual discount rate. adjusted by the number of periods per year to get the effective discount rate.
* stakerFee: what share of the fees belongs to 8pay token stakers (has a directly proportional effect on the outcome)
* mid: middle of the transaction count S-curve - when the exponential growth regime changes into the saturation regime
* slope: determines the speed of growth and saturation of the transaction count S-curve
* limit: determine the maximum saturation of the transaction count S-curve
* avgTransactionValue: average value of an 8pay transaction in USD (also has a directly proportional effect on the overall outcome)

In [17]:
def valuation(timeHorizon,\
              periodsPerYear,\
              discountRate,\
              stakerFee,\
              mid,slope,limit,\
              avgTransactionValue,\
             ):
    discountRate = numpy.power(1+discountRate/100, 1/periodsPerYear)
    stakerFee = stakerFee/100
    transactionCount = lambda t: logistic(t, mid, slope, limit)
    periodTransactionCount = numpy.zeros(periodsPerYear*timeHorizon)
    periodTransactionVolume = numpy.zeros(periodsPerYear*timeHorizon)
    periodCashflow = numpy.zeros(periodsPerYear*timeHorizon)
    periodDiscounted = numpy.zeros(periodsPerYear*timeHorizon)
    timeAxis = numpy.zeros(periodsPerYear*timeHorizon)
    for i in range(periodsPerYear*timeHorizon):
        timeAxis[i] = i/periodsPerYear
        periodTransactionCount[i] = average(transactionCount, [i/periodsPerYear, (i+1)/periodsPerYear])
        periodCashflow[i] = periodTransactionCount[i]*avgTransactionValue*stakerFee
        periodDiscounted[i] = periodCashflow[i]/numpy.power(discountRate,i+1)
        #print(f'time: {timeAxis[i]}, count: {periodTransactionCount[i]}, cashflow: {periodCashflow[i]}, discounted: {periodDiscounted[i]}')
    summedDiscounted = int(numpy.round(numpy.sum(periodDiscounted)/1000))
    discountedHorizon = int(numpy.round(periodDiscounted[-1]/(discountRate*(discountRate-1)*1000)))
    plt.plot(timeAxis, periodTransactionCount, )
    plt.xlabel('Years')
    plt.ylabel('Annualized Rate')
    plt.title('Transaction Rate')
    plt.grid(True)
    plt.show()
    plt.plot(timeAxis, periodCashflow, 'ob', timeAxis, periodDiscounted, '+r')
    plt.xlabel('Years')
    plt.ylabel('USD')
    plt.title('Cashflows (real and discounted)')
    plt.grid(True)
    plt.show()
    print(f'sum of discounted cashflows: $ {summedDiscounted}k')
    print(f'sum of horizon: $ {discountedHorizon}k')
    printmd(f'total net present value: **$ {summedDiscounted+discountedHorizon}k**')

In [18]:
interact(valuation,\
         timeHorizon = widgets.IntSlider(min=1,max=30,step=1,value=15,continuous_update=False),\
         periodsPerYear = widgets.SelectionSlider(options=[1,4, 12, 52],value=4),\
         discountRate = widgets.FloatSlider(min=0.1,max=25.1,step=0.1,value=5,continuous_update=False),\
         stakerFee = widgets.FloatSlider(min=0.1,max=0.9,step=0.05,value=0.5,continuous_update=False),\
         mid=widgets.FloatSlider(min=0,max=30,step=0.1,value=5,continuous_update=False),\
         slope=widgets.FloatSlider(min=0,max=3,step=0.1,value=1,continuous_update=False),\
         limit=widgets.IntText(value=12000000,continuous_update=False),\
         avgTransactionValue = widgets.IntSlider(min=1,max=30,step=1,value=5,continuous_update=False)
        )

interactive(children=(IntSlider(value=15, continuous_update=False, description='timeHorizon', max=30, min=1), …

<function __main__.valuation(timeHorizon, periodsPerYear, discountRate, stakerFee, mid, slope, limit, avgTransactionValue)>